In [50]:
# Calculation of complete reporting rates with program and stock data

%matplotlib inline  
# !pip install matplotlib

import pandas as pd
import numpy as np
import pandas_highcharts.core
from sqlalchemy import create_engine
import psycopg2
import matplotlib.pyplot 

# weird construct - if you don't from * import * you have to use datetime.datetime in commands
from datetime import datetime, date, timedelta

from isoweek import Week

import os

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "IMAM.settings")

from django.conf import settings
from home.management.commands.load_data import rename_cols, assign_state_lga_num

import django
django.setup()

# is this line necessary?
from home.models import Program, Stock, Site

In [51]:
# Import Program Data
engine = create_engine(
    'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{NAME}'.format(**settings.DATABASES['default']))
df = pd.read_sql_query("select * from program;", con=engine)


df['year_weeknum'] = zip(df['year'], df['weeknum'])
df['iso_year_weeknum'] = df['year_weeknum'].map(lambda x: Week(x[0], x[1]))

year, week, _ = date.today().isocalendar()
current_week = Week(year, week)

# since how many week this report is about
df['since_x_weeks'] = df['iso_year_weeknum'].map(lambda x: current_week - x)

In [77]:
active_sites = df.query('since_x_weeks<=8').query('siteid>101110001').groupby(['siteid', 'type'])['siteid'].unique()
active_sites

siteid      type
201110007   OTP      [201110007]
201110009   OTP      [201110009]
201110021   OTP      [201110021]
201110026   OTP      [201110026]
201110029   OTP      [201110029]
202110019   OTP      [202110019]
202110021   OTP      [202110021]
202110027   OTP      [202110027]
202110035   OTP      [202110035]
202110068   OTP      [202110068]
202610001   SC       [202610001]
203110002   OTP      [203110002]
203110037   OTP      [203110037]
203110038   OTP      [203110038]
203110043   OTP      [203110043]
203110053   OTP      [203110053]
204110006   OTP      [204110006]
204110014   OTP      [204110014]
204110026   OTP      [204110026]
204110029   OTP      [204110029]
204110035   OTP      [204110035]
205110025   OTP      [205110025]
205110039   OTP      [205110039]
206110004   OTP      [206110004]
206110011   OTP      [206110011]
206110020   OTP      [206110020]
206110032   OTP      [206110032]
206110034   OTP      [206110034]
207110055   OTP      [207110055]
207110082   OTP      [2071

In [75]:
all_sites = df.query('siteid>101110001').groupby(['siteid', 'type'])['siteid'].unique()
all_sites

siteid      type
201110007   OTP      [201110007]
201110009   OTP      [201110009]
201110021   OTP      [201110021]
201110026   OTP      [201110026]
201110029   OTP      [201110029]
201410014   OTP      [201410014]
202110019   OTP      [202110019]
202110021   OTP      [202110021]
202110027   OTP      [202110027]
202110035   OTP      [202110035]
202110068   OTP      [202110068]
202610001   SC       [202610001]
203110002   OTP      [203110002]
203110037   OTP      [203110037]
203110038   OTP      [203110038]
203110043   OTP      [203110043]
203110053   OTP      [203110053]
204110006   OTP      [204110006]
204110014   OTP      [204110014]
204110026   OTP      [204110026]
204110029   OTP      [204110029]
204110035   OTP      [204110035]
205110025   OTP      [205110025]
205110030   OTP      [205110030]
205110039   OTP      [205110039]
206110004   OTP      [206110004]
206110011   OTP      [206110011]
206110020   OTP      [206110020]
206110032   OTP      [206110032]
206110034   OTP      [2061

In [118]:
a = all_sites.to_frame()
a

,,siteid
siteid,type,
201110007,OTP,[201110007]
201110009,OTP,[201110009]
201110021,OTP,[201110021]
201110026,OTP,[201110026]
201110029,OTP,[201110029]
201410014,OTP,[201410014]
202110019,OTP,[202110019]
202110021,OTP,[202110021]
202110027,OTP,[202110027]


In [119]:
a = a.drop('siteid', axis=1).reset_index()
a

,siteid,type
0,201110007,OTP
1,201110009,OTP
2,201110021,OTP
3,201110026,OTP
4,201110029,OTP
5,201410014,OTP
6,202110019,OTP
7,202110021,OTP
8,202110027,OTP
9,202110035,OTP


In [104]:
b = active_sites.to_frame().drop('siteid', axis=1).reset_index()
b

,siteid,type
0,201110007,OTP
1,201110009,OTP
2,201110021,OTP
3,201110026,OTP
4,201110029,OTP
5,202110019,OTP
6,202110021,OTP
7,202110027,OTP
8,202110035,OTP
9,202110068,OTP


In [122]:
zip(a['siteid'], a['type'])

[(201110007, u'OTP'),
 (201110009, u'OTP'),
 (201110021, u'OTP'),
 (201110026, u'OTP'),
 (201110029, u'OTP'),
 (201410014, u'OTP'),
 (202110019, u'OTP'),
 (202110021, u'OTP'),
 (202110027, u'OTP'),
 (202110035, u'OTP'),
 (202110068, u'OTP'),
 (202610001, u'SC'),
 (203110002, u'OTP'),
 (203110037, u'OTP'),
 (203110038, u'OTP'),
 (203110043, u'OTP'),
 (203110053, u'OTP'),
 (204110006, u'OTP'),
 (204110014, u'OTP'),
 (204110026, u'OTP'),
 (204110029, u'OTP'),
 (204110035, u'OTP'),
 (205110025, u'OTP'),
 (205110030, u'OTP'),
 (205110039, u'OTP'),
 (206110004, u'OTP'),
 (206110011, u'OTP'),
 (206110020, u'OTP'),
 (206110032, u'OTP'),
 (206110034, u'OTP'),
 (207110015, u'OTP'),
 (207110055, u'OTP'),
 (207110082, u'OTP'),
 (207110088, u'OTP'),
 (207110089, u'OTP'),
 (207140015, u'OTP'),
 (208110006, u'OTP'),
 (208110021, u'OTP'),
 (208110024, u'OTP'),
 (208110029, u'OTP'),
 (208110033, u'OTP'),
 (209110005, u'OTP'),
 (209110009, u'OTP'),
 (209110011, u'OTP'),
 (209110019, u'OTP'),
 (209110025

In [121]:
len(set(zip(a['siteid'], a['type'])) - set(zip(b['siteid'], b['type'])))

89

In [113]:
b[b.apply(lambda x: x.values not in a.values, axis=1)]

,siteid,type


In [112]:
pd.concat([a,b,b]).drop_duplicates(keep=False)

,index,siteid,type
0,0.0,NaN,OTP
1,1.0,NaN,OTP
2,2.0,NaN,OTP
3,3.0,NaN,OTP
4,4.0,NaN,OTP
5,5.0,NaN,OTP
6,6.0,NaN,OTP
7,7.0,NaN,OTP
8,8.0,NaN,OTP
9,9.0,NaN,OTP


In [81]:
[x for x in all_sites]

[array([201110007]),
 array([201110009]),
 array([201110021]),
 array([201110026]),
 array([201110029]),
 array([201410014]),
 array([202110019]),
 array([202110021]),
 array([202110027]),
 array([202110035]),
 array([202110068]),
 array([202610001]),
 array([203110002]),
 array([203110037]),
 array([203110038]),
 array([203110043]),
 array([203110053]),
 array([204110006]),
 array([204110014]),
 array([204110026]),
 array([204110029]),
 array([204110035]),
 array([205110025]),
 array([205110030]),
 array([205110039]),
 array([206110004]),
 array([206110011]),
 array([206110020]),
 array([206110032]),
 array([206110034]),
 array([207110015]),
 array([207110055]),
 array([207110082]),
 array([207110088]),
 array([207110089]),
 array([207140015]),
 array([208110006]),
 array([208110021]),
 array([208110024]),
 array([208110029]),
 array([208110033]),
 array([209110005]),
 array([209110009]),
 array([209110011]),
 array([209110019]),
 array([209110025]),
 array([210110016]),
 array([21011

In [83]:
len(set([x[0] for x in all_sites]) - set([x[0] for x in active_sites]))

77

In [85]:
len(all_sites) - len(active_sites)

89

In [84]:
len(set([x[0] for x in all_sites]))

899

In [52]:
df.query('since_x_weeks>0').query('since_x_weeks<=8').query('siteid>101110001').groupby(['siteid', 'type'])['weeknum'].unique()


siteid      type
201110007   OTP     [12, 11, 13, 18, 17, 16, 15, 14]
201110009   OTP     [13, 12, 11, 14, 15, 17, 16, 18]
201110021   OTP                             [12, 11]
201110026   OTP                             [11, 12]
201110029   OTP                             [11, 12]
202110019   OTP             [17, 16, 15, 13, 12, 11]
202110021   OTP         [12, 11, 15, 14, 13, 16, 17]
202110027   OTP                             [11, 12]
202110035   OTP         [11, 14, 13, 12, 17, 16, 15]
202110068   OTP     [16, 11, 15, 14, 17, 12, 13, 18]
202610001   SC      [11, 18, 17, 16, 15, 14, 13, 12]
203110002   OTP     [13, 15, 14, 12, 11, 16, 18, 17]
203110037   OTP     [12, 15, 14, 17, 16, 13, 11, 18]
203110038   OTP     [12, 11, 13, 17, 16, 15, 14, 18]
203110043   OTP     [13, 12, 15, 14, 11, 17, 16, 18]
203110053   OTP             [13, 11, 12, 16, 14, 15]
204110006   OTP     [16, 12, 11, 14, 13, 15, 17, 18]
204110014   OTP             [15, 12, 11, 16, 13, 14]
204110026   OTP         [12, 

In [69]:
report = df.query('since_x_weeks>0').query('since_x_weeks<=8').query('siteid>101110001').groupby(['siteid', 'type'])['weeknum'].unique().map(
    lambda x: list(sorted(set(range(week - 8, week)) - set(x)))).to_frame()

report

,,weeknum
siteid,type,
201110007,OTP,[]
201110009,OTP,[]
201110021,OTP,"[13, 14, 15, 16, 17, 18]"
201110026,OTP,"[13, 14, 15, 16, 17, 18]"
201110029,OTP,"[13, 14, 15, 16, 17, 18]"
202110019,OTP,"[14, 18]"
202110021,OTP,[18]
202110027,OTP,"[13, 14, 15, 16, 17, 18]"
202110035,OTP,[18]


In [54]:
report.reset_index()

,siteid,type,weeknum
0,201110007,OTP,[]
1,201110009,OTP,[]
2,201110021,OTP,"[13, 14, 15, 16, 17, 18]"
3,201110026,OTP,"[13, 14, 15, 16, 17, 18]"
4,201110029,OTP,"[13, 14, 15, 16, 17, 18]"
5,202110019,OTP,"[14, 18]"
6,202110021,OTP,[18]
7,202110027,OTP,"[13, 14, 15, 16, 17, 18]"
8,202110035,OTP,[18]
9,202110068,OTP,[]


In [55]:
contacts = pd.read_sql_query("select * from registration;", con=engine)

In [56]:
contacts

,contact_uuid,urn,name,groups,siteid,type,first_seen,last_seen,post,mail,lga_num,state_num
0,935b9363-1db3-431c-ab7c-1fffaa44819b,+2348035946219,Sunusi Abubakar.,,1703110032,OTP,2016-09-06 09:24:51.750264,2017-05-11 12:40:24.883847,Community Health Officer,sunusiwurno@yahoo.com,1703.0,17.0
1,6afd0807-35a0-4707-8f9c-52bd137fd5fe,+2348080440504,Ibrahim Inuwa Lano.,,16,None,2016-09-01 11:23:27.667349,2017-05-11 12:40:03.315506,Stocks Manager,None,NaN,16.0
2,cb10403c-06d3-4965-92d6-06736aa890f8,+2348108130945,Bala Bande D.,,2103110001,OTP,2016-11-11 08:39:40.164187,2017-05-11 12:35:16.551243,In Charge Hospital/PHC,None,2103.0,21.0
3,9b407705-f5db-4819-98a9-121ab41867ce,+2348038489130,Balaraba Magaji.,,2106110020,OTP,2016-11-16 10:37:47.292656,2017-05-11 12:34:19.856747,Community Health Officer,None,2106.0,21.0
4,17ec92bb-b715-4391-9506-331310b65dfb,+2348063446207,1724110008.,,1724110008,OTP,2016-09-08 09:07:40.585498,2017-05-11 12:32:13.565853,Community Health Officer,None,1724.0,17.0
5,1d425221-b5a4-47e9-91df-b896868db4a6,+2348069142125,Abdulkadir Mohammad.,,2003110011,OTP,2016-10-31 19:14:07.344084,2017-05-11 12:21:57.659004,In Charge Hospital/PHC,akyasore@gmail.com,2003.0,20.0
6,b830de47-a791-4632-b729-761c966993a4,+2348025770177,Madu Salihu Yirnggau.,,818110010,OTP,2017-04-27 10:52:18.669483,2017-05-11 12:20:07.535285,In Charge Hospital/PHC,None,818.0,8.0
7,58cb9480-0994-46f7-a287-dd123ee7e91c,+2348137974197,Adama Haruna.,,3514110031,OTP,2016-10-08 09:59:15.711311,2017-05-11 12:18:12.845568,Community Health Officer,None,3514.0,35.0
8,c1fd47eb-493f-4687-9d32-fd04dce026b3,+2347064831294,Muhd Inuwa Ibrahim.,,1714110022,OTP,2016-09-06 09:24:42.840470,2017-05-11 12:13:48.610495,In Charge Hospital/PHC,mi2jhn@gmail.com,1714.0,17.0
9,7f6b75d7-fb31-4338-9a98-6ca141ee2e42,+2347013493266,Binta M Attahiru.,,3607210005,OTP,2016-11-07 11:03:01.242145,2017-05-11 12:10:10.238965,In Charge Hospital/PHC,None,3607.0,36.0


In [57]:
reminders = pd.merge(report.reset_index(), contacts, on=['siteid', 'type'])
reminders

,siteid,type,weeknum,contact_uuid,urn,name,groups,first_seen,last_seen,post,mail,lga_num,state_num
0,201110007,OTP,[],70eaa1cc-40ee-4323-aed3-647767bffcda,+2348134107585,Gracegidado.,,2016-10-25 09:10:31.153335,2017-05-11 08:43:28.567013,Community Health Officer,None,201.0,2.0
1,201110007,OTP,[],c004920d-9463-4176-a672-ce4332583b9d,+2349021245510,Assurance David.,,2016-10-25 09:05:47.222224,2017-05-11 08:43:26.656001,Community Health Officer,None,201.0,2.0
2,201110009,OTP,[],3857d121-cfef-4b64-afb6-74022862f6ea,+2348160018409,Jasper Gift Jonathan.,,2016-10-25 09:07:25.708751,2017-05-11 08:43:32.652318,Community Health Officer,None,201.0,2.0
3,201110009,OTP,[],1a4b900e-803e-43bb-a786-cf8e60435155,+2348165753101,Visia Ma Ambidi.,,2016-10-25 09:15:14.888164,2017-05-11 08:43:34.733916,Community Health Officer,None,201.0,2.0
4,201110021,OTP,"[13, 14, 15, 16, 17, 18]",ecb54d20-74cd-471d-a4a7-0f0208b88bdd,+2348092914506,Elcy Jemuel.,,2016-10-25 09:20:54.250857,2017-05-11 08:43:36.883325,Community Health Officer,None,201.0,2.0
5,201110021,OTP,"[13, 14, 15, 16, 17, 18]",7a9cdd08-35d3-481e-a2c1-12999890a6dd,+2349091141475,Regina Nathan.,,2016-10-25 09:13:06.611017,2017-05-11 08:43:38.881312,Community Health Officer,None,201.0,2.0
6,201110026,OTP,"[13, 14, 15, 16, 17, 18]",8ffd3195-a53c-4069-b394-ab665a0a64d0,+2348029967848,Mary Chindo.,,2016-10-25 09:04:08.197066,2017-05-11 08:43:43.118522,Community Health Officer,None,201.0,2.0
7,201110026,OTP,"[13, 14, 15, 16, 17, 18]",e890a957-cf10-4da0-b538-8a03a6c7844f,+2348022022110,Eglah Jotham Sam.,,2016-10-25 09:02:38.219964,2017-05-11 08:43:40.933403,Community Health Officer,None,201.0,2.0
8,201110029,OTP,"[13, 14, 15, 16, 17, 18]",681f91d1-ca59-4d32-bdd5-0081357bcf54,+2347080871500,Chaumasom Delon.,,2016-10-25 09:33:31.488528,2017-05-11 08:43:45.264680,Community Health Officer,None,201.0,2.0
9,201110029,OTP,"[13, 14, 15, 16, 17, 18]",ba1f390c-8a0d-4cba-b588-43b6bec664c8,+2349023158231,Peace Penuel.,,2016-10-25 09:05:39.510253,2017-05-11 08:43:47.487285,In Charge Hospital/PHC,None,201.0,2.0


In [58]:
reminders['number_of_missing_week'] = reminders['weeknum'].map(len)
reminders

,siteid,type,weeknum,contact_uuid,urn,name,groups,first_seen,last_seen,post,mail,lga_num,state_num,number_of_missing_week
0,201110007,OTP,[],70eaa1cc-40ee-4323-aed3-647767bffcda,+2348134107585,Gracegidado.,,2016-10-25 09:10:31.153335,2017-05-11 08:43:28.567013,Community Health Officer,None,201.0,2.0,0
1,201110007,OTP,[],c004920d-9463-4176-a672-ce4332583b9d,+2349021245510,Assurance David.,,2016-10-25 09:05:47.222224,2017-05-11 08:43:26.656001,Community Health Officer,None,201.0,2.0,0
2,201110009,OTP,[],3857d121-cfef-4b64-afb6-74022862f6ea,+2348160018409,Jasper Gift Jonathan.,,2016-10-25 09:07:25.708751,2017-05-11 08:43:32.652318,Community Health Officer,None,201.0,2.0,0
3,201110009,OTP,[],1a4b900e-803e-43bb-a786-cf8e60435155,+2348165753101,Visia Ma Ambidi.,,2016-10-25 09:15:14.888164,2017-05-11 08:43:34.733916,Community Health Officer,None,201.0,2.0,0
4,201110021,OTP,"[13, 14, 15, 16, 17, 18]",ecb54d20-74cd-471d-a4a7-0f0208b88bdd,+2348092914506,Elcy Jemuel.,,2016-10-25 09:20:54.250857,2017-05-11 08:43:36.883325,Community Health Officer,None,201.0,2.0,6
5,201110021,OTP,"[13, 14, 15, 16, 17, 18]",7a9cdd08-35d3-481e-a2c1-12999890a6dd,+2349091141475,Regina Nathan.,,2016-10-25 09:13:06.611017,2017-05-11 08:43:38.881312,Community Health Officer,None,201.0,2.0,6
6,201110026,OTP,"[13, 14, 15, 16, 17, 18]",8ffd3195-a53c-4069-b394-ab665a0a64d0,+2348029967848,Mary Chindo.,,2016-10-25 09:04:08.197066,2017-05-11 08:43:43.118522,Community Health Officer,None,201.0,2.0,6
7,201110026,OTP,"[13, 14, 15, 16, 17, 18]",e890a957-cf10-4da0-b538-8a03a6c7844f,+2348022022110,Eglah Jotham Sam.,,2016-10-25 09:02:38.219964,2017-05-11 08:43:40.933403,Community Health Officer,None,201.0,2.0,6
8,201110029,OTP,"[13, 14, 15, 16, 17, 18]",681f91d1-ca59-4d32-bdd5-0081357bcf54,+2347080871500,Chaumasom Delon.,,2016-10-25 09:33:31.488528,2017-05-11 08:43:45.264680,Community Health Officer,None,201.0,2.0,6
9,201110029,OTP,"[13, 14, 15, 16, 17, 18]",ba1f390c-8a0d-4cba-b588-43b6bec664c8,+2349023158231,Peace Penuel.,,2016-10-25 09:05:39.510253,2017-05-11 08:43:47.487285,In Charge Hospital/PHC,None,201.0,2.0,6


In [59]:
reminders.query('number_of_missing_week > 0')

,siteid,type,weeknum,contact_uuid,urn,name,groups,first_seen,last_seen,post,mail,lga_num,state_num,number_of_missing_week
4,201110021,OTP,"[13, 14, 15, 16, 17, 18]",ecb54d20-74cd-471d-a4a7-0f0208b88bdd,+2348092914506,Elcy Jemuel.,,2016-10-25 09:20:54.250857,2017-05-11 08:43:36.883325,Community Health Officer,None,201.0,2.0,6
5,201110021,OTP,"[13, 14, 15, 16, 17, 18]",7a9cdd08-35d3-481e-a2c1-12999890a6dd,+2349091141475,Regina Nathan.,,2016-10-25 09:13:06.611017,2017-05-11 08:43:38.881312,Community Health Officer,None,201.0,2.0,6
6,201110026,OTP,"[13, 14, 15, 16, 17, 18]",8ffd3195-a53c-4069-b394-ab665a0a64d0,+2348029967848,Mary Chindo.,,2016-10-25 09:04:08.197066,2017-05-11 08:43:43.118522,Community Health Officer,None,201.0,2.0,6
7,201110026,OTP,"[13, 14, 15, 16, 17, 18]",e890a957-cf10-4da0-b538-8a03a6c7844f,+2348022022110,Eglah Jotham Sam.,,2016-10-25 09:02:38.219964,2017-05-11 08:43:40.933403,Community Health Officer,None,201.0,2.0,6
8,201110029,OTP,"[13, 14, 15, 16, 17, 18]",681f91d1-ca59-4d32-bdd5-0081357bcf54,+2347080871500,Chaumasom Delon.,,2016-10-25 09:33:31.488528,2017-05-11 08:43:45.264680,Community Health Officer,None,201.0,2.0,6
9,201110029,OTP,"[13, 14, 15, 16, 17, 18]",ba1f390c-8a0d-4cba-b588-43b6bec664c8,+2349023158231,Peace Penuel.,,2016-10-25 09:05:39.510253,2017-05-11 08:43:47.487285,In Charge Hospital/PHC,None,201.0,2.0,6
10,202110019,OTP,"[14, 18]",b679ca25-db75-4137-90e5-8dfbc56bfef2,+2348086762757,Ahmed Aminu.,,2016-10-25 09:21:41.934179,2017-05-11 08:43:51.790589,Community Health Officer,ahmedaminu2044@yahoo.com,202.0,2.0,2
11,202110019,OTP,"[14, 18]",bd5385d4-5953-4a65-829d-1ffd25ae3bb9,+2347033992688,Noel Norah.,,2016-10-25 09:03:02.263195,2017-05-11 08:43:53.780110,Technical Assistance,None,202.0,2.0,2
12,202110021,OTP,[18],2ab2d7c2-b1a5-4cec-857a-a0d06ddaf776,+2348103586218,Salihu Bakura.,,2016-10-25 09:04:29.325309,2017-05-11 08:44:00.064687,Community Health Officer,None,202.0,2.0,1
13,202110021,OTP,[18],4e09ed3b-3131-4dd0-833d-2f0b6b72e98d,+2348088482226,Salihu Bakura.,,2016-10-25 11:16:24.094473,2017-05-11 08:44:02.325080,Community Health Officer,None,202.0,2.0,1


In [60]:
set([tuple(sorted(x)) for x in reminders.query('number_of_missing_week > 0').weeknum])

{(11,),
 (11, 12, 13),
 (11, 12, 13, 14),
 (11, 12, 13, 14, 15, 16),
 (11, 12, 13, 14, 15, 16, 17),
 (11, 12, 13, 14, 15, 18),
 (11, 12, 13, 14, 17, 18),
 (11, 12, 13, 15),
 (11, 12, 13, 17),
 (11, 12, 13, 17, 18),
 (11, 12, 14, 15, 16, 17, 18),
 (11, 12, 15, 16, 17, 18),
 (11, 12, 17, 18),
 (11, 12, 18),
 (11, 13, 18),
 (11, 14, 15, 16, 17, 18),
 (11, 15, 16, 17, 18),
 (11, 17, 18),
 (11, 18),
 (12, 13, 14),
 (12, 13, 14, 15, 16, 17, 18),
 (12, 13, 14, 17, 18),
 (12, 13, 16, 17, 18),
 (12, 14, 16, 17, 18),
 (12, 16),
 (12, 18),
 (13,),
 (13, 14, 15, 16, 17, 18),
 (13, 15, 16, 17, 18),
 (13, 16, 17, 18),
 (13, 17, 18),
 (13, 18),
 (14,),
 (14, 15, 16, 17, 18),
 (14, 15, 18),
 (14, 16, 17, 18),
 (14, 17, 18),
 (14, 18),
 (15,),
 (15, 16, 17, 18),
 (15, 17, 18),
 (16,),
 (16, 17, 18),
 (16, 18),
 (17,),
 (17, 18),
 (18,)}